In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Metabolic_Rate"
cohort = "GSE26440"

# Input paths
in_trait_dir = "../../input/GEO/Metabolic_Rate"
in_cohort_dir = "../../input/GEO/Metabolic_Rate/GSE26440"

# Output paths
out_data_file = "../../output/preprocess/Metabolic_Rate/GSE26440.csv"
out_gene_data_file = "../../output/preprocess/Metabolic_Rate/gene_data/GSE26440.csv"
out_clinical_data_file = "../../output/preprocess/Metabolic_Rate/clinical_data/GSE26440.csv"
json_path = "../../output/preprocess/Metabolic_Rate/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE26440_family.soft.gz', 'GSE26440_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Metabolic_Rate/GSE26440/GSE26440_family.soft.gz
Matrix file: ../../input/GEO/Metabolic_Rate/GSE26440/GSE26440_series_matrix.txt.gz


Background Information:
!Series_title	"Expression data for derivation of septic shock subgroups"
!Series_summary	"Background:  Septic shock is a heterogeneous syndrome within which probably exist several biological subclasses. Discovery and identification of septic shock subclasses could provide the foundation for the design of more specifically targeted therapies. Herein we tested the hypothesis that pediatric septic shock subclasses can be discovered through genome-wide expression profiling. Methods:  Genome-wide expression profiling was conducted using whole blood-derived RNA from 98 children with septic shock, followed by a series of bioinformatic approaches targeted at subclass discovery and characterization. Results:  Three putative subclasses (subclasses A, B, and C) were initially identified based on an empiric, discovery-oriented expression filter and unsupervised hierarchical clustering. Statistical comparison of the 3 putative subclasses (ANOVA, Bonferonni correction, p < 0.

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import re
from typing import Optional, Dict, Any, Callable

# 1. Determine gene expression data availability
# Based on series title and summary, this appears to be gene expression data from whole blood
is_gene_available = True

# 2.1 Identify data availability in sample characteristics
# Trait (Metabolic Rate) - not directly available, but we can use "outcome" as a proxy for metabolic rate
# since the study examines septic shock subgroups which relate to metabolic pathways
trait_row = 3  # "outcome" row

# Age data is available
age_row = 2  # "age (years)" row

# Gender data is not available in the sample characteristics
gender_row = None

# 2.2 Data type conversion functions
def convert_trait(value: str) -> Optional[float]:
    """Convert outcome data to a binary representation."""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    match = re.search(r'outcome:\s*(.*)', value)
    if not match:
        return None
    
    outcome = match.group(1).strip().lower()
    
    # Convert outcome to binary: Survivor = 0, Nonsurvivor = 1
    if outcome == 'survivor':
        return 0.0
    elif outcome == 'nonsurvivor':
        return 1.0
    else:
        return None  # n/a cases

def convert_age(value: str) -> Optional[float]:
    """Convert age data to continuous values."""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    match = re.search(r'age \(years\):\s*(.*)', value)
    if not match:
        return None
    
    try:
        age = float(match.group(1).strip())
        return age
    except ValueError:
        return None

def convert_gender(value: str) -> Optional[float]:
    """Convert gender data to binary (0=female, 1=male)."""
    # Gender data is not available
    return None

# 3. Save metadata about cohort usability
# Initial filtering - check if trait and gene expression data are available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical feature extraction (only if trait_row is not None)
if trait_row is not None:
    # The clinical data has been extracted in a previous step
    # The sample characteristics dictionary shows the structure of the data
    
    # Create a simple DataFrame from the sample characteristics dictionary
    # The structure is {row_index: [list of unique values]}
    unique_values = {
        0: ['tissue: whole blood'],
        1: ['disease state: septic shock patient', 'disease state: normal control'],
        2: ['age (years): 9.4', 'age (years): 3', 'age (years): 2.1', 'age (years): 1.9', 'age (years): 0.8', 
            'age (years): 0.3', 'age (years): 9.1', 'age (years): 0.6', 'age (years): 2.6', 'age (years): 8.8', 
            'age (years): 0.9', 'age (years): 4.5', 'age (years): 2', 'age (years): 0.5', 'age (years): 2.4', 
            'age (years): 0.1', 'age (years): 2.7', 'age (years): 6.8', 'age (years): 7.1', 'age (years): 7.3', 
            'age (years): 4.4', 'age (years): 2.8', 'age (years): 3.7', 'age (years): 0.7', 'age (years): 0', 
            'age (years): 1.6', 'age (years): 1.1', 'age (years): 2.2', 'age (years): 4.2', 'age (years): 2.3'],
        3: ['outcome: Nonsurvivor', 'outcome: Survivor', 'outcome: n/a'],
        4: ['group: A', 'group: C', 'group: B', 'group: n/a']
    }
    
    # Load the clinical data from a previous step
    # This should be a matrix where each row is a characteristic and each column is a sample
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    
    if os.path.exists(clinical_data_path):
        clinical_data = pd.read_csv(clinical_data_path, index_col=0)
        
        # Use the library function to extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of extracted clinical features:")
        print(preview)
        
        # Create the output directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the selected clinical features to CSV
        selected_clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to: {out_clinical_data_file}")
    else:
        print(f"Clinical data file not found at: {clinical_data_path}")
        print("This step depends on clinical data being extracted in a previous step.")


Clinical data file not found at: ../../input/GEO/Metabolic_Rate/GSE26440/clinical_data.csv
This step depends on clinical data being extracted in a previous step.


### Step 3: Initial Data Loading

In [4]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE26440_family.soft.gz', 'GSE26440_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Metabolic_Rate/GSE26440/GSE26440_family.soft.gz
Matrix file: ../../input/GEO/Metabolic_Rate/GSE26440/GSE26440_series_matrix.txt.gz
Background Information:
!Series_title	"Expression data for derivation of septic shock subgroups"
!Series_summary	"Background:  Septic shock is a heterogeneous syndrome within which probably exist several biological subclasses. Discovery and identification of septic shock subclasses could provide the foundation for the design of more specifically targeted therapies. Herein we tested the hypothesis that pediatric septic shock subclasses can be discovered through genome-wide expression profiling. Methods:  Genome-wide expression profiling was conducted using whole blood-derived RNA from 98 children with septic shock, followed by a series of bioinformatic approaches targeted at subclass discovery and characterization. Results:  Three putative subclass

### Step 4: Dataset Analysis and Clinical Feature Extraction

In [5]:
import os
import pandas as pd
import json
import numpy as np
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Metabolic_Rate) - From the background, we can infer metabolic rate through septic shock status
trait_row = 1  # 'disease state: septic shock patient' or 'disease state: normal control'

# Age - Age information is available in row 2
age_row = 2  # 'age (years): X.X'

# Gender - Not explicitly available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> Optional[int]:
    """Convert disease state to binary trait value (septic shock = 1, normal control = 0)."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if "septic shock" in value.lower():
        return 1
    elif "normal control" in value.lower():
        return 0
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age string to continuous numeric value."""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (female = 0, male = 1)."""
    # Since gender information is not available, this function is a placeholder
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial validation and filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
if trait_row is not None:
    try:
        # Assuming clinical_data was created in a previous step
        # We'll use the function from the library to extract clinical features
        from tools.preprocess import get_feature_data
        
        # Create a simple DataFrame from the sample characteristics we've already seen
        sample_chars = {
            1: ['disease state: septic shock patient', 'disease state: normal control'],
            2: ['age (years): 9.4', 'age (years): 3', 'age (years): 2.1', 'age (years): 1.9', 
                'age (years): 0.8', 'age (years): 0.3', 'age (years): 9.1', 'age (years): 0.6', 
                'age (years): 2.6', 'age (years): 8.8', 'age (years): 0.9', 'age (years): 4.5', 
                'age (years): 2', 'age (years): 0.5', 'age (years): 2.4', 'age (years): 0.1', 
                'age (years): 2.7', 'age (years): 6.8', 'age (years): 7.1', 'age (years): 7.3', 
                'age (years): 4.4', 'age (years): 2.8', 'age (years): 3.7', 'age (years): 0.7', 
                'age (years): 0', 'age (years): 1.6', 'age (years): 1.1', 'age (years): 2.2', 
                'age (years): 4.2', 'age (years): 2.3']
        }
        
        # Convert to DataFrame format that geo_select_clinical_features expects
        clinical_data = pd.DataFrame(sample_chars)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted features
        preview = preview_df(clinical_features)
        print("Clinical Features Preview:")
        print(preview)
        
        # Save to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        # If there's an error, we'll mark trait data as unavailable
        is_trait_available = False
        
        # Update the validation with the correct trait availability
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available
        )


Error processing clinical data: All arrays must be of the same length


### Step 5: Gene Data Extraction

In [6]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:


No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 54675
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 6: Gene Identifier Review

In [7]:
# Looking at the gene identifiers format
# These identifiers in the format 'XXXXXX_at', '1007_s_at', etc. are Affymetrix probe IDs
# These are not human gene symbols but probe identifiers from Affymetrix microarrays
# They will need to be mapped to standard gene symbols for cross-platform compatibility

requires_gene_mapping = True


### Step 7: Gene Annotation

In [8]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 8: Gene Identifier Mapping

In [9]:
# 1. Decide which keys in the gene annotation dataframe correspond to gene identifiers and gene symbols
# Looking at the gene annotation dictionary:
# - 'ID' contains probe identifiers like '1007_s_at', '1053_at' which match the gene expression data
# - 'Gene Symbol' contains the human gene symbols like 'DDR1 /// MIR4640', 'RFC2'

# 2. Get the gene mapping dataframe by extracting the relevant columns
# The ID column contains the probe identifiers that match the gene expression data's index
# The Gene Symbol column contains the corresponding gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This handles the many-to-many relation between probes and genes by:
# - Dividing probe expression equally among mapped genes
# - Summing up all probe contributions for each gene
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=gene_mapping)

# Print information about the resulting gene expression dataframe
print(f"Gene expression data converted from {len(gene_data)} probes to {len(gene_data.index.unique())} unique genes")
print("First 10 gene symbols in the processed gene data:")
print(gene_data.index[:10])


Gene expression data converted from 21278 probes to 21278 unique genes
First 10 gene symbols in the processed gene data:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 9: Data Normalization and Linking

In [10]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Re-examine the clinical data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print out the sample characteristics to verify available rows
characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample characteristics dictionary:")
print(characteristics_dict)

# Define conversion functions for the clinical features based on the actual data
def convert_trait(value):
    """Convert outcome data to a binary representation for septic shock survival."""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if 'survivor' == value:
        return 0.0  # Survived
    elif 'nonsurvivor' == value:
        return 1.0  # Did not survive
    else:
        return None  # n/a cases

def convert_age(value):
    """Convert age data to continuous values."""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    match = re.search(r'age \(years\):\s*(.*)', value)
    if not match:
        return None
    
    try:
        age = float(match.group(1).strip())
        return age
    except (ValueError, TypeError):
        return None

# Create the clinical dataframe using the correct row indices based on sample characteristics
try:
    # Row 3 contains outcome (survivor/nonsurvivor) - this is our trait of interest
    # Row 2 contains age information
    clinical_df = geo_select_clinical_features(
        clinical_data,
        trait="Metabolic_Rate",  # Using this as the trait name as per variable definition
        trait_row=3,  # Outcome (survival) as the trait
        convert_trait=convert_trait,
        gender_row=None,  # No gender information available
        convert_gender=None,
        age_row=2,     # Age information in row 2
        convert_age=convert_age
    )
    
    print("Clinical data preview:")
    print(preview_df(clinical_df.T))  # Transpose for better viewing
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Saved clinical data to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data_normalized)
    print(f"Shape of linked data: {linked_data.shape}")
    
    # 4. Handle missing values in the linked data
    linked_data_cleaned = handle_missing_values(linked_data, 'Metabolic_Rate')
    print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")
    
    # 5. Check if the trait and demographic features are biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, 'Metabolic_Rate')
    
    # 6. Validate the dataset and save cohort information
    note = "Dataset contains gene expression data from whole blood of pediatric septic shock patients. The trait variable is survival (0=survivor, 1=nonsurvivor). The study identifies gene expression patterns associated with different septic shock subclasses (A, B, C) that have different survival rates."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note=note
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Final linked data not saved.")
        
except Exception as e:
    print(f"Error in processing clinical data: {e}")
    # If we failed to extract clinical data, update the cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=None,
        df=pd.DataFrame(),
        note="Failed to extract clinical data. Gene expression data is available but missing trait information."
    )
    print("Dataset validation failed due to missing clinical data. Only gene data saved.")

Shape of gene data after normalization: (19845, 130)


Saved normalized gene data to ../../output/preprocess/Metabolic_Rate/gene_data/GSE26440.csv
Sample characteristics dictionary:
{0: ['tissue: whole blood'], 1: ['disease state: septic shock patient', 'disease state: normal control'], 2: ['age (years): 9.4', 'age (years): 3', 'age (years): 2.1', 'age (years): 1.9', 'age (years): 0.8', 'age (years): 0.3', 'age (years): 9.1', 'age (years): 0.6', 'age (years): 2.6', 'age (years): 8.8', 'age (years): 0.9', 'age (years): 4.5', 'age (years): 2', 'age (years): 0.5', 'age (years): 2.4', 'age (years): 0.1', 'age (years): 2.7', 'age (years): 6.8', 'age (years): 7.1', 'age (years): 7.3', 'age (years): 4.4', 'age (years): 2.8', 'age (years): 3.7', 'age (years): 0.7', 'age (years): 0', 'age (years): 1.6', 'age (years): 1.1', 'age (years): 2.2', 'age (years): 4.2', 'age (years): 2.3'], 3: ['outcome: Nonsurvivor', 'outcome: Survivor', 'outcome: n/a'], 4: ['group: A', 'group: C', 'group: B', 'group: n/a']}
Clinical data preview:
{'Metabolic_Rate': [1.0,

Shape of linked data after handling missing values: (116, 19847)
For the feature 'Metabolic_Rate', the least common label is '1.0' with 17 occurrences. This represents 14.66% of the dataset.
The distribution of the feature 'Metabolic_Rate' in this dataset is fine.

Quartiles for 'Age':
  25%: 0.8
  50% (Median): 2.05
  75%: 5.75
Min: 0.0
Max: 10.9
The distribution of the feature 'Age' in this dataset is fine.



Saved processed linked data to ../../output/preprocess/Metabolic_Rate/GSE26440.csv
